In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

#### PCA

Principal Component Analysis (Метод главных компонент)

In [2]:
# столбец 0 - количество комнат
# столбец 1 - площадь дома (в тысячах квадратных футов)
houses = np.array([[5, 1.6],
                   [4, 1.4],
                   [6, 1.9],
                   [3, 1.1],
                   [4, 1.25]])

In [3]:
plt.scatter(houses[:, 0], houses[:, 1])
plt.xlabel('Количество комнат')
plt.ylabel('Площадь дома')

По графику видно, что два признака сильно скоррелированы и 

их можно расположить вдоль одной прямой,

то есть фактически сжать данные до одного измерения.

<img src='1d.svg' align='left'></img>

Это означает, что данные могут быть представлены одной главной компонентой

с минимальной потерей информации.

#### Алгоритм PCA

In [4]:
houses

Найдем среднее значение каждого признака

In [5]:
mean_values = np.mean(houses, axis=0)
mean_values

Отнимем от значений признаков средние значения

In [6]:
houses_centered = houses - mean_values
houses_centered

Найдем матрицу ковариаций

(матрица ковариаций - матрица, состоящая из попарных ковариаций признаков)

In [7]:
covarience_matrix = np.cov(houses_centered.T)
covarience_matrix

In [8]:
# Проверка правильности найденной матрицы - найдем дисперсии центрированных признаков,
# они должны совападать по значению с главной диагональю ковариационной матрицы
print(np.var(houses_centered[:, 0], ddof=1))
print(np.var(houses_centered[:, 1], ddof=1))

Найдем собственные значения (eigenvalues) 

и собственные векторы (eigenvectors) матрицы ковариаций

In [9]:
eigenvalues, eigenvectors = np.linalg.eig(covarience_matrix)

In [10]:
eigenvalues

In [11]:
eigenvectors

Собственные значения (eigenvalues) сильно отличаются - 

eigenvalues[0] намного больше, чем eigenvalues[1],

поэтому оставим только собственный вектор из столбца 0 массива eigenvectors,

то есть eigenvectors[:, 0].

Умножим центрированные данные на этот вектор.

In [12]:
result_pre = np.dot(houses_centered, eigenvectors[:, 0])
result = result_pre.reshape(-1, 1)
result

Сравнение с результатом вычислений в библиотеке sklearn

In [13]:
from sklearn.decomposition import PCA

In [14]:
pca = PCA(n_components=1)

In [15]:
mc = pca.fit_transform(houses)
mc

In [16]:
pca.explained_variance_ratio_

#### Применение PCA в моделях машинного обучения

Загрузим сохраненные нами данные по пассажирам Титаника

In [17]:
X_train = pd.read_pickle('X_train.pkl')
y_train = pd.read_pickle('y_train.pkl')

In [ ]:
X_valid = pd.read_pickle('X_valid.pkl')
y_valid = pd.read_pickle('y_valid.pkl')

In [ ]:
X_train.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler(with_mean=False)

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_valid_scaled = pd.DataFrame(X_valid_scaled, columns=X_train.columns)

Сначала мы не будем указывать число компонент (по умолчанию все компоненты будут сохранены)

In [ ]:
pca = PCA(random_state=100)

In [ ]:
pca.fit(X_train_scaled)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
np.sum(pca.explained_variance_ratio_[:7])

Оставим только 7 главных компонент

In [ ]:
pca = PCA(n_components=7, random_state=100)

In [ ]:
X_train_mc = pca.fit_transform(X_train_scaled)

In [ ]:
X_valid_mc = pca.transform(X_valid_scaled)

Построим модель логистической регрессии

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train_mc, y_train)

In [ ]:
y_pred = lr.predict(X_valid_mc)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_valid, y_pred)